In [5]:
import pickle
import pandas as pd
import os

In [73]:
# names = ['../triplets_sci/lemmas_triplets.pickle',
#          '../triplets_sci/numbers_triplets.pickle',
#           ]

names = ['../triplets_env/lemmas_triplets.pickle',
         '../triplets_env/numbers_triplets.pickle',
          ]

total = []
for name in names:
    with open(name, 'rb') as f:
        ls = pickle.load(f)
        print(len(ls))
        total += ls

28055
25772


In [74]:
len((total))

53827

In [75]:
def collect_mix_nodes(total_ls, out_path):

    ls = []
    for triplet in set(total):
        elem = {}
        elem["children"] = triplet[2]
        elem["parents"] = [triplet[0], triplet[1]]
        elem["grandparents"] = None
        elem["case"] = "simple_triplet_2parent"
        elem["1parent_def"] = triplet[0]
        elem["2parent_def"] = triplet[1]
    
        ls.append(elem)

    with open(out_path, 'wb') as f:
        pickle.dump(ls, f)

In [76]:
collect_mix_nodes(total, '../data/env/eurovoc/raw_pairs/mix_nodes_lemmas.pickle')

In [77]:
with open('../data/env/eurovoc/raw_pairs/mix_nodes_lemmas.pickle', 'rb') as f:
    ls = pickle.load(f)

In [78]:
len(ls)

31066

In [28]:
ls = []
for triplet in set(total):
    elem = {}
    elem["children"] = triplet[2]
    elem["parents"] = [triplet[0], triplet[1]]
    elem["grandparents"] = None
    elem["case"] = "simple_triplet_2parent"
    elem["1parent_def"] = triplet[0]
    elem["2parent_def"] = triplet[1]

ls.append(elem)

In [29]:
elem

{'children': 'industrial sociology',
 'parents': ['industrial chemistry', 'ethnology'],
 'grandparents': None,
 'case': 'simple_triplet_2parent',
 '1parent_def': 'industrial chemistry',
 '2parent_def': 'ethnology'}

In [53]:

def clean_elem(elem, keys_to_remove_digits=["children"]):
    removes = set(keys_to_remove_digits)
    if not "changed" in elem.keys():
        for field in ["children", "parents", "grandparents", "brothers"]:
            if field in elem.keys():
                elem[field] = delete_techniqal(
                    elem[field], remove=True
                )  # (field in removes))
                elem["changed"] = True
    return elem


def delete_techniqal(elem, remove):
    if isinstance(elem, str):
        if ".n." in elem and remove:
            return elem.split(".")[0].replace("_", " ")
        else:
            return elem.replace("_", " ")

    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word, remove))
        return new_words

clean = clean_elem(elem, keys_to_remove_digits=["children"])
transformed_term = (
            "first hypernym: "
            + clean["parents"][0]
            + " ("
            + elem["1parent_def"]
            + "), second hypernym: "
            + clean["parents"][1]
            + " ("
            + elem["2parent_def"]
            + ") | hyponyms:"
        )

In [54]:
transformed_term

'first hypernym: industrial chemistry (industrial chemistry), second hypernym: ethnology (ethnology) | hyponyms:'

In [55]:
def get_term(s):
    term = s.split("|")[-2]
    term = term.split(":")
    if len(term) == 2:
        return term[-1].strip()
    else:
        first_parent = term[1].split(' (')[0].strip()
        second_parent = term[2].split('(')[0].strip()

        return first_parent + '_' + second_parent

In [57]:
get_term(transformed_term)

'industrial chemistry_ethnology'

In [41]:
s = ' industrial chemistry (industrial chemistry), second hypernym'

In [42]:
s.split('(')

[' industrial chemistry ', 'industrial chemistry), second hypernym']

In [47]:
with open('../data/env/eurovoc/raw_pairs/pred_hypernym_lemmas.pickle', 'rb') as f:
    ph = pickle.load(f)

In [48]:
ph[0]

{'children': 'climate change policy',
 'parents': 'adaptation to climate change',
 'grandparents': None,
 'case': 'predict_hypernym',
 'child_def': 'climate change policy'}

In [50]:
clean = clean_elem(ph[0], keys_to_remove_digits=["parents"])

transformed_term = (
    "hyponym: " + clean["children"] + " (" + ph[0]["child_def"] + ") | hypernym:"
)

In [52]:
get_term(transformed_term)

'climate change policy (climate change policy)'